# Project 1: Assembling Genes

   <div class="alert alert-block alert-danger">
    <center>Due: <b>Monday, September 5, 8:59pm</b>.</center> 
   </div>
   
   <div class="alert alert-block alert-warning">
   <center>
       <b>Collaboration and Resource Policy</b>
    </center>
    
For this assignment, you are encouraged to work with one other person satisfying the constraints from Class 2. 
You are permitted (actually _encouraged_) to discuss these problems with anyone you want, including other students in the class. If you do discuss the specific questions in the assignment with anyone other than your assignment partner and the course staff, though, you should list them in the _External resources used_ section below.
    
You are welcome to use any resources you want for this assignment, other than ones that would defeat the purpose of the assignment. This means you should not look at answers or code from previous semesters of this course, or from any other students in the class (other than your collaboration with your partner), and if you find code that implements the problem you are being asked to do for the assignment, you should not use that code. You should document all external resource you use that are not part of the course materials in the _External resources used_ section below.
    </div>

**Team submitting this assignment:**  
<div class="alert alert-success">
    <b><em>list each member of your team here, including both your name and UVA computing id</em></b>

Team Members (Names):  

Team Member UVA Computing IDs:

</div>

**External resources used:** 
<div class="alert alert-success">
<em>It is not necessary to list the course materials, but if you used any other resources, including discussing problems with students not on your team, list them here.</em>
    
External Resources Used:

</div>

In this project, we will explore genome assembly—the process of determining the order of nucleotides in DNA from fragmented reads. As you might have studied in the reading assignments, genome assembly can get quite complicated, as problems such as full sequence coverage, finding a good length for reads (the $k$ in $k$-mer), and sequencing errors present challenges for sequencing analysis and accuracy. You can assume perfect coverage for all parts of the assignment and no read errors for the first two questions.


<b>Submission</b>: Please submit the code you wrote to generate your answers for all parts using this form: <a href="https://forms.gle/rNTXfYojTLEQ8idg6"><em>https://forms.gle/rNTXfYojTLEQ8idg6</em></a>. Your answers should be in the Jupyter Notebook, along with your code. Before submission, you should make a copy of your notebook file with the name <i>uvaid1\_uvaid2.ipynb</i> (where <i>uvaidn</i> is each teammates UVA id) so the submitted file identifies you. You and your partner should submit a single file once together. Submission is due 8:59 pm (EST) on Monday, September 5.

## Install basic required packages.

- Install basic required packages, should be run only once. You may need to restart the kernel after this stage.
- Make sure you have [graphviz](https://graphviz.org/download/) installed on your system.
- The second cell adds Graphviz to your path, you may have to change based on where the install folder is.

<b>NOTE: We provide utils.py, which may contain helpful functions for you to use, as well as gvmagic.py, which is a deprecated package to use graphviz within the notebook</b>

In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'

## Genome Assembly

For this part, you're given reads generated while trying to sequence the DNA of a TeleTubby (some unknown organism) with a \textit{very} small genetic code. By answering the following questions, you will learn how to assemble the original genome sequence from sequence reads.

Sequencing data is often stored in FASTQ file format. In TeleTubby.fastq, you will find the data organized in a particular order that repeats every four lines. The first line contains the metadata that encodes the name of the read, the experiment type, the kind of sequencing machine used, etc. The second line is the sequence of bases. The third line functions as a placeholder line. The fourth line is a sequence of base qualities that encode the qualities for the corresponding bases in the sequence line. We will only work with the sequence and quality score lines in this question.

In [4]:
import collections
import matplotlib.pyplot as plt
import numpy as np
import utils
from tqdm import tqdm

#### Question 1.1.1 GC-content

The GC-content (or the ratio of G and C nucleotides) is related to the melting temperature of the DNA double helix. Use the following equation to calculate the melting temperature of DNA for TeleTubby $t_m$ in Celsius:

\begin{equation*}
t_m = 64.9+0.41(\%GC)-\frac{500}{\text{length of sequence}}
\end{equation*}

As a reference, the human genome is known to have between 35%-60% GC-content. 

In [5]:
import utils

# Read sequence reads (error-free) from file
sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')

# Calculate %GC content
sequence_length = 0
GC_content = 0
for i in range(len(sequence_reads)):
    sequence_length += len(sequence_reads[i])
    for nucleotide in sequence_reads[i]:
        if nucleotide == "G" or nucleotide == "C":
            GC_content += 1
GC_content = GC_content / sequence_length * 100 

# Print out temperature in Celsius
tm = 64.9 + 0.41*GC_content - 500/sequence_length
print(tm)

84.34709897610922


#### Question 1.1.2 Interpreting quality scores

Phred33 quality scores are represented as the character with an ASCII code equal to its value + 33 (to make them easy to print alongside genome sequences). List the top 5 most frequent scores in ASCII symbol as well as their Phredd33 scores in TeleTubby.fastq. You can refer to the [official Illumina website](https://support.illumina.com/help/BaseSpace_OLH_009008/Content/Source/Informatics/BS/QualityScoreEncoding_swBS.htm) to reference the scoring encoding.

What is the average Phred33 score in TeleTubby.fastq?

In [6]:
# Calculate and print average Phred33 score
char_dictionary = {}

sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')

for i in range(len(qualities)):
    for score in qualities[i]:
        if score in char_dictionary:
            char_dictionary[score][1] += 1
        else:
            char_dictionary[score] = [ord(score)-33, 1]

# Average calculation
sum_of_scores = 0
num_of_scores = 0
for char in char_dictionary.keys():
    sum_of_scores += char_dictionary[char][0] * char_dictionary[char][1]
    num_of_scores += char_dictionary[char][1]

average = sum_of_scores / num_of_scores
print("Average Phred33 Score:")
print(average)

# Top 5 
count = 0
print("Top 5 Most Frequent Scores:")
while count < 5:
    frequent = ["", 0, 0]
    for char in char_dictionary.keys():
        if char_dictionary[char][1] > frequent[1]:
            frequent[0] = char
            frequent[1] = char_dictionary[char][1]
            frequent[2] = char_dictionary[char][0]
            char_dictionary[char][1] = -1
    print(frequent[0], frequent[2])
    count += 1

Average Phred33 Score:
34.476535836177476
Top 5 Most Frequent Scores:
5 20
D 35
F 37
O 46
< 27


#### Question 1.1.3 Frequency analysis

Looking at repetitions in the sequence can be helpful in estimating the "redudancy" in the organisms. Humand and other evolved animals have a lot of redundancy, while smaller organisms like bacteria have highly packed genomes. One heuristic to estimate this before actually performing the assembly could be looking at how often certain $k$-mers are repeated.

<b>Print out the 3 most frequent k-mers with their frequencies</b>

In [7]:
# Find and print out the three most repeated k-mers and their frequencies
sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')

kmer_dict = {}

for read in sequence_reads:
    if read in kmer_dict:
        kmer_dict[read] += 1
    else:
        kmer_dict[read] = 1
        
count = 0
while count < 3:
    frequent = ["", 0]
    for kmer in kmer_dict.keys():
        if kmer_dict[kmer] > frequent[1]:
            frequent[0] = kmer
            frequent[1] = kmer_dict[kmer]
            kmer_dict[kmer] = -1
    print(frequent)
    count += 1     

['GCTATCGC', 3]
['TATCGCAA', 2]
['CTATCGCA', 2]


### Question 1.2. Greedy approach

One of the approaches to assemble the genome from the given reads is a greedy algorithm. Have a look at the greedy algorithm described on [Wikipedia](https://en.wikipedia.org/wiki/Sequence_assembly#Greedy_algorithm) and answer the following.

#### Question 1.2.1 What would the runtime be of this algorithm, given $n$ $k$-mer reads?

<i>Answer</i>:

#### Question 1.2.2 Would this algorithm always yield a unique solution?

<i>Answer</i>:

#### Question 1.2.3 Would this algorithm always yield the <i>right</i> solution?

<i>Answer</i>:

### Question 1.3 Graph-based approaches

Graphs for genome assembly can be constructed in two ways:

- de Bruijn graph: Processing $k-$mers as edges, with $(k-1)-$mers as nodes, and
- Overlap graph: Processing $k-$mers as nodes, with $(k-1)-$mers as edges.

de Bruijn graphs can be processed to find Euler paths, while Overlap graphs can be processed to find Hamiltonian paths. Both of these are valid ways to reconstruct the original genome.

<b>Use one of these two techniques to reconstruct the sequence, and print out your reconstructed sequence. Which method did you pick out of the two, and why? (hint: imagine what would happen when we have millions of reads). Use the k-mers provided in TeleTubby.fastq</b>.

We provide some skeleton code that you may use, but you may also come up with your own solution.

In [8]:
# Read reads into graph

def build_graph(k_mers):
    edges = []
    nodes = set()
    matrix = {}
    outdegrees = {}
    indegrees = {}
    # Your code here
    for sequence in sequence_reads:
        edges.append(sequence)
        prefix = sequence[:-1]
        suffix = sequence[1:]
        nodes.add(prefix)
        nodes.add(suffix)

        if prefix not in matrix:
            matrix[prefix] = {}
            outdegrees[prefix] = 0

        if suffix not in indegrees:
            indegrees[suffix] = 0

        # Adds to adjacency matrix
        matrix[prefix][suffix] = sequence
        outdegrees[prefix] += 1
        indegrees[suffix] += 1
        
            
    return nodes, edges, matrix, outdegrees, indegrees

nodes, edges, matrix, outdegrees, indegrees = build_graph(sequence_reads)

In [9]:
# Identify start node: outgoing edges is more than incoming
start = None
end = None

for subseq in nodes:
    diff = outdegrees.get(subseq,0) - indegrees.get(subseq, 0)
    if diff == 1:
        start = subseq
    elif diff == -1:
        end = subseq

In [47]:
# Hierholzer's algorithm
visited = set()
path = []

def dfs(e, v):
    v_edges = list(matrix.get(v, {}).values())
    v_edges_unvisited = [_ for _ in v_edges if _ not in visited]
    while (v_edges_unvisited):
        edge = v_edges_unvisited.pop()
        visited.add(edge)
        v2 = edge[1:]
        dfs(edge, v2)
    
    if e:
        path.append(e)

In [48]:
dfs(None, start)

print(len(path))
print("Missing from path: ", set(edges) ^ set(path)) # Should be empty

288
Missing from path:  set()


In [49]:
# Print assembled sequence
print("Sequence of k mers")
path

Sequence of k mers


['GCAATGTT',
 'TGCAATGT',
 'ATGCAATG',
 'GATGCAAT',
 'CGATGCAA',
 'TCGATGCA',
 'TTCGATGC',
 'CTTCGATG',
 'CCTTCGAT',
 'CCCTTCGA',
 'GCCCTTCG',
 'CGCCCTTC',
 'CCGCCCTT',
 'GCCGCCCT',
 'TGCCGCCC',
 'GTGCCGCC',
 'GGTGCCGC',
 'TGGTGCCG',
 'CTGGTGCC',
 'ACTGGTGC',
 'TACTGGTG',
 'CTACTGGT',
 'GCTACTGG',
 'CGCTACTG',
 'TCGCTACT',
 'ATCGCTAC',
 'TATCGCTA',
 'CGCTATCG',
 'GCGCTATC',
 'GGCGCTAT',
 'AGGCGCTA',
 'GAGGCGCT',
 'GGAGGCGC',
 'CGGAGGCG',
 'CCGGAGGC',
 'CCCGGAGG',
 'GCCCGGAG',
 'TGCCCGGA',
 'CTGCCCGG',
 'GCTGCCCG',
 'CGCTGCCC',
 'TCGCTGCC',
 'ATCGCTGC',
 'TATCGCTG',
 'CTATCGCT',
 'CCGCTATC',
 'TCCGCTAT',
 'ATCCGCTA',
 'CATCCGCT',
 'ACATCCGC',
 'CACATCCG',
 'GCACATCC',
 'TGCACATC',
 'GTGCACAT',
 'GGTGCACA',
 'TGGTGCAC',
 'ATGGTGCA',
 'TATGGTGC',
 'GTATGGTG',
 'AGTATGGT',
 'GAGTATGG',
 'GGAGTATG',
 'GGGAGTAT',
 'TGGGAGTA',
 'GTGGGAGT',
 'AGTGGGAG',
 'AAGTGGGA',
 'AAAGTGGG',
 'AAAAGTGG',
 'AAAAAGTG',
 'AAAAAAGT',
 'AAAAAAAG',
 'CAAAAAAA',
 'GCAAAAAA',
 'AGCAAAAA',
 'GAGCAAAA',
 'TGAGCAAA',

In [15]:
# Needed to properly visualize graphs
%load_ext gvmagic

In [16]:
# Helper function to visualize de Bruijn graphs
%dotstr utils.viz_debruijn(nodes_bruijn, edges_bruijn)

NameError: name 'nodes_bruijn' is not defined

In [ ]:
# Main assembly algorithm

def assemble_sequence(nodes, edges):
    assembled_sequence = ""
    # Your code here
    return assembled_sequence

In [ ]:
# Output assembled sequence

assmebled_seq = assemble_sequence(nodes, edges)

## Question 2 - Sequencing SARS-CoV-2 virus

Let's move on from TeleTubbies to real-world organisms. Let's start small- with a variant of the SARS-CoV-2 virus. You're given reads from <i>actual</i> genome sequencing runs in the SARS-CoV2.fastq file provided.

Repeat Question 1.3 on this data. You can re-use your implementation and simply run it on the new data. Print out your reconstructed sequence to a file "output.txt". For this part, we will still assume that all the reads are error-free. Set $k=25$.

In [ ]:
# Read sequence reads
sequence_reads_covid, qualities_covid = utils.read_fastq('SARS-CoV2.fastq')

In [ ]:
# Read reads into graph
nodes_covid, edges_covid = build_graph(sequence_reads_covid)

In [ ]:
# Call main assembly algorithm
assmebled_covid_seq = assemble_sequence(nodes_covid, edges_covid)

In [ ]:
# Write assembled sequence to file

assmebled_seq = "" # Use your assembled genome
with open("covid_overlap.txt", "w") as f:
    f.write(assmebled_seq)

# Question 3- Error-Aware Assembly (Extra Credit)

In the parts above, we assumed error-free reads while assembling $k$-mers. As much as we'd like that, actual reads can (and do) have errors, captured by their Phred scores. For this question, you're given raw, actual reads from sequencing runs (download reads here: https://sra-pub-sars-cov2.s3.amazonaws.com/sra-src/SRR11528307/ABS2-LN-R1_cleaned_paired.fastq.gz).  Given these reads and their Phred33 scores, can you assemble the genome?

<b>Print out your assembled sequence, along with a brief explanation of how your algorithm works</b>

This is an open-ended question. You are free to use any approach to deal with the issue. Make sure you provide your code, along with any assumptions you may have, in the cells below.